## 1) Creación de dataset entrenamiento

### Librerias

In [ ]:
#importo librerias
import glob
import os
from tqdm import tqdm


### Instalacion de pysatproc (correr solo una vez)

In [ ]:
#!pip instal pysatproc

### Variables para configuracion de satproc_extract_chips

In [ ]:

dates="2021-01-01_2021-03-01"                                      # nombre de la carpeta con las imagenes
zona = "zona_3"                                                    # zona con la que vamos a trabajar
version =  2                                                       # version de la corrida actual 
size = 100                                                         # tamaño (pixeles) de las imagenes que se van a generar
step_size = 50                                                     # tamaño de la ventana
vector_file_path = "../data/shp/gt/R3/Asentamiento_R3_v2.geojson"  # ruta y archivo con la verdad de campo

###### estas variables no modifican #####
path_to_files = os.path.join("../images-S2",zona,dates,"*.tif")    # ruta a la carpeta con las imagenes satelitales
vector_aoi_file_path = vector_file_path
output_folder = os.path.join("../dataset/data_train",zona,"v"+str(version),str(size)+"_"+str(step_size))         # Destination path <ndim>/<size>_<step-size>/


### Lista de imagenes que van a ser procesadas

In [ ]:
!ls $path_to_files

### Ayuda de satproc_extract_chips

In [ ]:
#satproc_extract_chips --help

### Generacion del dataset para entrenamiento

In [ ]:
#Creo el dataset para entrenamiento (imag + anotaciones)
!satproc_extract_chips \
    $path_to_files \
    -o  $output_folder \
    --size $size \
    --step-size $step_size \
    --aoi $vector_file_path \
    --labels $vector_file_path \
    --label-property 'class' \ # nombre del campo que tiene la clase a predecir
    --classes 'A' \            # valor que toma la clase a predecir
    --rescale \
    --rescale-mode s2_rgb_extra --lower-cut 1 --upper-cut 100


# 2) Entrenamiento

### Instalacion de unet (correr solo una vez)

In [ ]:
#!pip install unetseg

### Librerias

In [ ]:
#importo la red neuronal
from unetseg.train import TrainConfig, train
from unetseg.evaluate import plot_data_generator
import os

### Variables para configuracion de unet

In [ ]:
spe=100          # step per epoch (debe ser multipo de 16 y estar entre 80 y 320)
size_unet=160    # tamaño en la red (img/batch_size 10%)

config = TrainConfig(width=size_unet,  
                     height=size_unet, 
                     n_channels=4, 
                     n_classes=1,
                     apply_image_augmentation=True,
                     seed=42,
                     epochs=30, 
                     batch_size=16, 
                     steps_per_epoch=spe, 
                     early_stopping_patience=10,
                     validation_split=0.1, 
                     test_split=0.1,  
                     model_architecture='unet', #unetplusplus
                     images_path=os.path.join("../dataset/data_train",zona,'v'+str(version),str(size)+"_"+str(step_size)),                     
                     model_path=os.path.join('../data/weights/model',zona+'_UNet_techo_4D_spe'+str(spe)+'_img'+str(size)+'_size'+str(size)+'_sz'+str(step_size)+'.h5'),
                     evaluate=True ,
                     class_weights = [1]) 

### Grafico de 3 tiles al azar

In [ ]:
plot_data_generator(num_samples=3, fig_size=(10, 10), train_config = config,img_ch = 3)

### Corro el entrenamiento

In [ ]:
#Corro el modelo con los parametros establecidos
res_config = train(config)

### Graficos Loss y Mean_iou de train/test

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16,4))

plt.subplot(121)
plt.plot(res_config.history['loss'])
plt.plot(res_config.history['val_loss'])
plt.title('Loss')

plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')

plt.subplot(122)
plt.plot(res_config.history['mean_io_u'])
plt.plot(res_config.history['val_mean_io_u'])
plt.title('mean_iou')
plt.ylabel('val_mean_iou')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')


plt.show()